# IMDB Random Forests Regression
![ImdbIcon](../images/imdbheader.jpg)

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, AdaBoostRegressor
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix, plot_confusion_matrix, classification_report

C:\Users\nolan_fur2pfn\.conda\envs\dsi\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\nolan_fur2pfn\.conda\envs\dsi\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
C:\Users\nolan_fur2pfn\.conda\envs\dsi\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
  stacklevel=1)


In [2]:
totalscore_df = pd.read_csv('../data/totalscore_df.csv')

In [3]:
totalscore_df.columns

Index(['movie_title', 'year', 'actors', 'plot', 'duration', 'Action',
       'Adventure', 'Animation', 'Biography', 'Comedy', 'Crime', 'Documentary',
       'Drama', 'Family', 'Fantasy', 'History', 'Horror', 'Music', 'Mystery',
       'Romance', 'Sci-Fi', 'Sport', 'Thriller', 'War', 'Western', 'votes',
       'weighted_average_vote', 'total_votes', 'mean_vote', 'median_vote',
       'votes_1', 'votes_2', 'votes_3', 'votes_4', 'votes_5', 'votes_6',
       'votes_7', 'votes_8', 'votes_9', 'votes_10', 'us_voters_rating',
       'us_voters_votes', 'plot_sentiment', 'director_score', 'actor_score',
       'actress_score', 'imdb_score'],
      dtype='object')

## Random Forests Regression

In [189]:
features = ['year', 'duration', 'Action', 'Adventure', 'Animation', 'Biography', 'Comedy', 'Crime', 'Documentary',
       'Drama', 'Family', 'Fantasy', 'History', 'Horror', 'Music', 'Mystery',
       'Romance', 'Sci-Fi', 'Sport', 'Thriller', 'War', 'Western',
        'total_votes', 'plot_sentiment','votes_1', 'votes_2', 'votes_3', 'votes_4', 'votes_5', 'votes_6',
       'votes_7', 'director_score']

In [190]:
X = totalscore_df[features]
y = totalscore_df['imdb_score']

In [191]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, test_size = .50)

In [192]:
rf = RandomForestRegressor()

In [193]:
rf.fit(X_train,y_train)

RandomForestRegressor()

In [194]:
print("Training Score:", rf.score(X_train, y_train))
print(" ")
print("Testing Score:", rf.score(X_test, y_test))

Training Score: 0.9932937430888794
 
Testing Score: 0.9451351319780443


In [195]:
print("Cross Val Score:", cross_val_score(rf, X_train, y_train, cv = 5).mean())

Cross Val Score: 0.9439088291331291


In [196]:
model_params = {}
count = 0

In [201]:
rf_params = {
    'max_depth': [12, 14, 16],
    'min_samples_leaf': [1, 2, 3],
    'min_samples_split': [1, 2, 3],
    'n_estimators': [200, 300, 400],
}

gs = GridSearchCV(rf, param_grid=rf_params, cv = 5, n_jobs = 12)

gs.fit(X_train, y_train)

count += 1

gs.best_params_['best_score'] = gs.best_score_

gs.best_params_['training_score'] = gs.score(X_train, y_train)

gs.best_params_['testing_score'] = gs.score(X_test, y_test)

model_params[f'model_{count}'] = gs.best_params_

model_df = pd.DataFrame.from_dict(model_params, orient='index')

model_df

,max_depth,min_samples_leaf,min_samples_split,n_estimators,best_score,training_score,testing_score
model_1,4,2,3,100,0.747900,0.791172,0.740249
model_2,8,2,2,300,0.926291,0.971183,0.925709
model_3,8,2,2,200,0.926476,0.971018,0.924301
model_4,12,2,2,300,0.944202,0.989614,0.945748
model_5,16,2,2,300,0.944852,0.990722,0.947355


## AdaBoostRegressor

In [202]:
model_params_two = {}
count_two = 0

In [205]:
ada = AdaBoostRegressor(base_estimator=RandomForestRegressor())

ada_params = {
    'base_estimator__max_depth': [5, 6, 7],
    'learning_rate': [.90, .95, 1.],
    'n_estimators': [50, 60, 70],
}

gs_two = GridSearchCV(ada, param_grid=ada_params, cv=3, n_jobs = 12)

gs_two.fit(X_train, y_train)

count_two += 1

gs_two.best_params_['best_score'] = gs_two.best_score_

gs_two.best_params_['training_score'] = gs_two.score(X_train, y_train)

gs_two.best_params_['testing_score'] = gs_two.score(X_test, y_test)

model_params_two[f'model_{count_two}'] = gs_two.best_params_

model_df_two = pd.DataFrame.from_dict(model_params_two, orient='index')

model_df_two

,base_estimator__max_depth,learning_rate,n_estimators,best_score,training_score,testing_score
model_1,3,1.00,30,0.806001,0.844506,0.800649
model_2,5,1.00,50,0.915408,0.955964,0.920011
model_3,7,0.95,70,0.944595,0.986029,0.955077
